In [9]:
from anytree import Node, RenderTree

class CKYParser:
    def __init__(self, grammar):
        self.grammar = grammar

    def parse(self, sentence):
        words = sentence.split()
        n = len(words)
        parse_table = [[set() for _ in range(n)] for _ in range(n)]
        tree_table = [[[] for _ in range(n)] for _ in range(n)]

        # Initialize the table with the grammar rules for each word
        for j in range(n):
            for lhs, rhs in self.grammar.items():
                if words[j] in rhs:
                    parse_table[j][j].add(lhs)
                    tree_table[j][j].append(Node(lhs, word=words[j]))

        # Fill the parse table and tree table
        for length in range(2, n + 1):
            for start in range(n - length + 1):
                end = start + length - 1
                for mid in range(start, end):
                    for lhs, productions in self.grammar.items():
                        for production in productions:
                            if len(production) == 2:
                                left, right = production
                                if left in parse_table[start][mid] and right in parse_table[mid + 1][end]:
                                    parent = Node(lhs)  # Create the parent node
                                    
                                    for left_tree in tree_table[start][mid]:
                                        for right_tree in tree_table[mid + 1][end]:
                                            # Create a new parent node for the current production
                                            new_parent = Node(lhs)
                                            # Set left and right trees as children of the new parent
                                            left_tree.parent = new_parent  # Set the parent of left_tree
                                            right_tree.parent = new_parent  # Set the parent of right_tree
                                            tree_table[start][end].append(new_parent)

                                    parse_table[start][end].add(lhs)

        return parse_table, tree_table

    def print_parse_table(self, parse_table):
        for i in range(len(parse_table)):
            for j in range(len(parse_table)):
                if parse_table[i][j]:
                    print(f"Parse Table[{i}][{j}]: {parse_table[i][j]}")

    def print_parse_tree(self, tree_table):
        if tree_table[0][len(tree_table) - 1]:
            print("Parse Tree:")
            root = tree_table[0][len(tree_table) - 1][0]
            for pre, _, node in RenderTree(root):
                if hasattr(node, 'word'):  # Only access word attribute if it exists
                    print(f"{pre}{node.name} (word: {node.word})")
                else:
                    print(f"{pre}{node.name}")  # Print non-terminals without word
        else:
            print("No valid parse tree found.")

# Example grammar in CNF
grammar = {
    'S': {('NP', 'VP')},
    'NP': {('Det', 'Noun'), ('Pronoun',)},
    'VP': {('Verb', 'NP'), ('Verb', 'PP')},
    'PP': {('Preposition', 'NP')},
    'Det': {'a', 'the'},
    'Noun': {'cat', 'dog', 'man', 'telescope'},
    'Pronoun': {'I', 'you', 'he', 'she'},
    'Verb': {'saw', 'ate'},
    'Preposition': {'with', 'by'}
}

# Instantiate the CKY parser
cky_parser = CKYParser(grammar)

# Parse a sentence
sentence = "the cat saw a dog"
parse_table, tree_table = cky_parser.parse(sentence)

# Print the parse table
cky_parser.print_parse_table(parse_table)

# Print the parse tree
cky_parser.print_parse_tree(tree_table)


Parse Table[0][0]: {'Det'}
Parse Table[0][1]: {'NP'}
Parse Table[0][4]: {'S'}
Parse Table[1][1]: {'Noun'}
Parse Table[2][2]: {'Verb'}
Parse Table[2][4]: {'VP'}
Parse Table[3][3]: {'Det'}
Parse Table[3][4]: {'NP'}
Parse Table[4][4]: {'Noun'}
Parse Tree:
S
├── NP
│   ├── Det (word: the)
│   └── Noun (word: cat)
└── VP
    ├── Verb (word: saw)
    └── NP
        ├── Det (word: a)
        └── Noun (word: dog)
